In [1]:
import gmaps
import json
from ipywidgets.embed import embed_minimal_html

# Google developer API key
from api_keys import g_key

# Access maps with unique API key
# gmaps.configure(api_key=g_key)

# # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# # Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)


In [2]:
clean_city_data = pd.read_csv("output_data/weatherdata.csv")
clean_city_data.head()

,City,Latitude,Longitude,Max Temp (F),Humidity %,Cloudiness %,Wind Speed (mph),Country,Date
0,Port Lincoln,-34.73,135.87,60.278,62,25,10.200446,AU,1595989716
1,Ushuaia,-54.80,-68.30,35.600,80,40,20.803542,AR,1595989716
2,Cayenne,4.93,-52.33,78.800,88,26,12.504495,GF,1595989716
3,Treinta y Tres,-33.23,-54.38,36.212,84,0,5.883152,UY,1595989717
4,Longyearbyen,78.22,15.64,51.800,81,75,11.408394,SJ,1595989645


In [12]:
# create heatmap showing humidity. use lat and lng as locations and humidity as weight
locations = clean_city_data[['Latitude', 'Longitude']].astype(float)
humidity = clean_city_data['Humidity %'].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius = 3)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [ ]:
embed_minimal_html('heatmap.html', views=[fig])


In [4]:
clean_city_data.head()


,City,Latitude,Longitude,Max Temp (F),Humidity %,Cloudiness %,Wind Speed (mph),Country,Date
0,Port Lincoln,-34.73,135.87,60.278,62,25,10.200446,AU,1595989716
1,Ushuaia,-54.80,-68.30,35.600,80,40,20.803542,AR,1595989716
2,Cayenne,4.93,-52.33,78.800,88,26,12.504495,GF,1595989716
3,Treinta y Tres,-33.23,-54.38,36.212,84,0,5.883152,UY,1595989717
4,Longyearbyen,78.22,15.64,51.800,81,75,11.408394,SJ,1595989645


In [5]:
hotel_data = clean_city_data.loc[clean_city_data["Max Temp (F)"] < 80].\
            loc[clean_city_data["Max Temp (F)"] > 70].\
            loc[clean_city_data["Cloudiness %"] == 0].\
            loc[clean_city_data["Wind Speed (mph)"] < 10].\
            loc[clean_city_data["Humidity %"] < 100].\
            dropna(how='any')
hotel_data["Hotel Name"] = ""
hotel_data

,City,Latitude,Longitude,Max Temp (F),Humidity %,Cloudiness %,Wind Speed (mph),Country,Date,Hotel Name
48,Balkhash,46.84,74.98,71.582,47,0,8.366156,KZ,1595989721,
52,Sousa,-6.76,-38.23,73.022,59,0,5.592350,BR,1595989722,
109,Arraial do Cabo,-22.97,-42.03,71.600,83,0,8.052984,BR,1595989582,
166,Varna,43.22,27.92,75.002,56,0,2.236940,BG,1595989558,
173,Kiryat Gat,31.61,34.76,78.008,58,0,1.006623,IL,1595989514,
190,Beyneu,45.32,55.20,71.294,39,0,9.864905,KZ,1595989738,
227,Almonte,37.26,-6.52,75.992,52,0,5.994999,ES,1595989743,
258,Araguaçu,-12.93,-49.83,72.878,55,0,5.435764,BR,1595989748,
266,Plácido de Castro,-10.34,-67.19,76.316,63,0,1.521119,BR,1595989749,
272,Taft,31.75,54.21,78.800,22,0,4.697574,IR,1595989749,


In [6]:
# loop through location of perfect weather conditions and use google api to find a nearby 
# hotel within 5000 meters. Add hotel name to hotel_data

target_radius = 5000
target_type = "lodging"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
   "keyword": "Hotel",
   "radius": target_radius,
   "type": target_type,
   "key": g_key
}
for index, row in hotel_data.iterrows():
   #get latitude from df
    lat = row["Latitude"]
   #get longitude from df
    lng = row["Longitude"]
    target_coord = f"{lat},{lng}"
    params["location"] = target_coord
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    try:
        hotel_data.loc[index, "Hotel Name"] = results[0]["name"]
    except IndexError:
        print("No hotel found nearby") 
hotel_data

No hotel found nearby
No hotel found nearby


,City,Latitude,Longitude,Max Temp (F),Humidity %,Cloudiness %,Wind Speed (mph),Country,Date,Hotel Name
48,Balkhash,46.84,74.98,71.582,47,0,8.366156,KZ,1595989721,Guest House Comfort-House-Balkhash
52,Sousa,-6.76,-38.23,73.022,59,0,5.592350,BR,1595989722,Hotel La Ville
109,Arraial do Cabo,-22.97,-42.03,71.600,83,0,8.052984,BR,1595989582,Orlanova Boutique Hotel
166,Varna,43.22,27.92,75.002,56,0,2.236940,BG,1595989558,Hotel Odesos
173,Kiryat Gat,31.61,34.76,78.008,58,0,1.006623,IL,1595989514,Desert Gat
190,Beyneu,45.32,55.20,71.294,39,0,9.864905,KZ,1595989738,"HOTEL ""AKNUR"""
227,Almonte,37.26,-6.52,75.992,52,0,5.994999,ES,1595989743,Camino Del Mar
258,Araguaçu,-12.93,-49.83,72.878,55,0,5.435764,BR,1595989748,Brunas Park Hotel
266,Plácido de Castro,-10.34,-67.19,76.316,63,0,1.521119,BR,1595989749,Hotel Ferreira
272,Taft,31.75,54.21,78.800,22,0,4.697574,IR,1595989749,abibi hostel


In [7]:
# drop any location with no nearby hotel
hotel_data = hotel_data[hotel_data["Hotel Name"] != '']
hotel_data

,City,Latitude,Longitude,Max Temp (F),Humidity %,Cloudiness %,Wind Speed (mph),Country,Date,Hotel Name
48,Balkhash,46.84,74.98,71.582,47,0,8.366156,KZ,1595989721,Guest House Comfort-House-Balkhash
52,Sousa,-6.76,-38.23,73.022,59,0,5.592350,BR,1595989722,Hotel La Ville
109,Arraial do Cabo,-22.97,-42.03,71.600,83,0,8.052984,BR,1595989582,Orlanova Boutique Hotel
166,Varna,43.22,27.92,75.002,56,0,2.236940,BG,1595989558,Hotel Odesos
173,Kiryat Gat,31.61,34.76,78.008,58,0,1.006623,IL,1595989514,Desert Gat
190,Beyneu,45.32,55.20,71.294,39,0,9.864905,KZ,1595989738,"HOTEL ""AKNUR"""
227,Almonte,37.26,-6.52,75.992,52,0,5.994999,ES,1595989743,Camino Del Mar
258,Araguaçu,-12.93,-49.83,72.878,55,0,5.435764,BR,1595989748,Brunas Park Hotel
266,Plácido de Castro,-10.34,-67.19,76.316,63,0,1.521119,BR,1595989749,Hotel Ferreira
272,Taft,31.75,54.21,78.800,22,0,4.697574,IR,1595989749,abibi hostel


In [8]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]
locations = hotel_data[["Latitude", "Longitude"]]

In [9]:
# put hotel name, city and country into a list to add to marker on heatmap
columns = hotel_data[["Hotel Name", "City", "Country"]]
hotel_info = columns.values.tolist()
hotel_info

[['Guest House Comfort-House-Balkhash', 'Balkhash', 'KZ'],
 ['Hotel La Ville', 'Sousa', 'BR'],
 ['Orlanova Boutique Hotel', 'Arraial do Cabo', 'BR'],
 ['Hotel Odesos', 'Varna', 'BG'],
 ['Desert Gat', 'Kiryat Gat', 'IL'],
 ['HOTEL "AKNUR"', 'Beyneu', 'KZ'],
 ['Camino Del Mar', 'Almonte', 'ES'],
 ['Brunas Park Hotel', 'Araguaçu', 'BR'],
 ['Hotel Ferreira', 'Plácido de Castro', 'BR'],
 ['abibi hostel', 'Taft', 'IR'],
 ['Villa Elisio Hotel & Spa', 'Naples', 'IT'],
 ['Norsemen Inn', 'Camrose', 'CA'],
 ['Resort Safari Souria منتجع سفاري سورية', 'Salamiyah', 'SY'],
 ['CONFIANCE HOTEL', 'Crateús', 'BR'],
 ['Cullera Holiday Hotel', 'Cullera', 'ES'],
 ['Grand Hotel', 'Ziarat', 'PK'],
 ['Prospector Inn', 'Flin Flon', 'CA'],
 ['Veneza Plaza Hotel', 'Gurupi', 'BR'],
 ['Bela Vista Hotel', 'Lavras da Mangabeira', 'BR'],
 ['Hotel El Alamo', 'Lora del Río', 'ES'],
 ['Pefkaki Boutique Hotel Loutraki', 'Corinth', 'GR']]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
            info_box_content=[f"Hotel: <b>{hotel[0]}</b> <br>" +
                              f"City: <b>{hotel[1]}</b> <br>" + 
                              f"Country: <b>{hotel[2]}</b>" for hotel in hotel_info])
fig = gmaps.figure()
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
embed_minimal_html('export.html', views=[fig])
